## Deliverable 2. Create a Customer Travel Destinations Map.

In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

from ipywidgets.embed import embed_minimal_html

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,46.26,76,90,1.99,overcast clouds
1,1,Butaritari,KI,3.0707,172.7902,82.33,81,100,15.86,light rain
2,2,Mataura,NZ,-46.1927,168.8643,49.48,97,100,3.49,light rain
3,3,Kurilsk,RU,45.2333,147.8833,42.78,96,100,14.25,overcast clouds
4,4,Rikitea,PF,-23.1203,-134.9692,77.00,85,72,19.01,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Butaritari,KI,3.0707,172.7902,82.33,81,100,15.86,light rain
4,4,Rikitea,PF,-23.1203,-134.9692,77.00,85,72,19.01,broken clouds
7,7,Atuona,PF,-9.8000,-139.0333,78.64,71,12,11.01,few clouds
10,10,Georgetown,MY,5.4112,100.3354,84.88,82,20,3.00,moderate rain
14,14,Statesboro,US,32.4488,-81.7832,81.93,61,90,5.75,overcast clouds
17,17,Mumford,GH,5.2618,-0.7590,82.72,76,100,8.95,overcast clouds
19,19,Lata,PT,40.1629,-8.3327,82.04,54,0,8.34,clear sky
21,21,Aksay,RU,47.2600,39.8700,80.82,70,92,6.89,light rain
22,22,Bandarbeyla,SO,9.4942,50.8122,80.11,84,7,28.23,clear sky
24,24,Avera,US,33.1940,-82.5271,82.90,40,76,14.12,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                228
City                   228
Country                228
Lat                    228
Lng                    228
Max Temp               228
Humidity               228
Cloudiness             228
Wind Speed             228
Current Description    228
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#Not applicable

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Butaritari,KI,82.33,light rain,3.0707,172.7902,
4,Rikitea,PF,77.00,broken clouds,-23.1203,-134.9692,
7,Atuona,PF,78.64,few clouds,-9.8000,-139.0333,
10,Georgetown,MY,84.88,moderate rain,5.4112,100.3354,
14,Statesboro,US,81.93,overcast clouds,32.4488,-81.7832,
17,Mumford,GH,82.72,overcast clouds,5.2618,-0.7590,
19,Lata,PT,82.04,clear sky,40.1629,-8.3327,
21,Aksay,RU,80.82,light rain,47.2600,39.8700,
22,Bandarbeyla,SO,80.11,clear sky,9.4942,50.8122,
24,Avera,US,82.90,broken clouds,33.1940,-82.5271,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
         hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
          print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
# 7. Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df.count()

City                   211
Country                211
Max Temp               211
Current Description    211
Lat                    211
Lng                    211
Hotel Name             211
dtype: int64

In [17]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))